In [ ]:
import requests
import lxml.html as lh

gdelt_base_url = 'http://data.gdeltproject.org/events/'
gdelt_gkg_url = 'http://api.gdeltproject.org/api/v1/gkg_geojson'
# get the list of all the links on the gdelt file page
page = requests.get(gdelt_base_url+'index.html')
doc = lh.fromstring(page.content)
link_list = doc.xpath("//*/ul/li/a/@href")

# separate out those links that begin with four digits 
file_list = [x for x in link_list if str.isdigit(x[0:4])]


In [ ]:
masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
directory = requests.get(masterListUrl)
results = directory.content.split('\n')

In [ ]:
results

In [ ]:
pd.options.display.max_rows = 200

In [ ]:
import pandas as pd

df = pd.DataFrame(data.json())

In [ ]:
df['coords'] = df.features.apply(lambda row: row['geometry']['coordinates'])
df['lat'] = df.features.apply(lambda row: row['geometry']['coordinates'][1])
df['lon'] = df.features.apply(lambda row: row['geometry']['coordinates'][0])
df['name'] = df.features.apply(lambda row: row['properties']['name'])
df['pubdate'] = df.features.apply(lambda row: row['properties']['urlpubtimedate'])
df['urltone'] = df.features.apply(lambda row: row['properties']['urltone'])
df['mentionedNames'] = df.features.apply(lambda row: row['properties']['mentionednames'])
df['mentioinedThemes'] = df.features.apply(lambda row: row['properties']['mentionedthemes'])
df['url'] = df.features.apply(lambda row: row['properties']['url'])

# GDELT 2.0 Access

In [ ]:
import requests
masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
directory = requests.get(masterListUrl)
clean = directory.content.split('\n')
clean = map(lambda x: x.split(' '),clean)

# Logic for GDELT module

Enter a date
* default is take current time and most recent file
* enter historical date; defaults to no time specificity
    * parse
    * add feature to enter time for historical and pull closest 15 minute file
    
choose a database
*  Select between events, event mentions or gkg

return it as a python or R dataframe
*  use the feather library for Python


## Code Pieces and Functions

In [ ]:
defaultDateEntry = ""
stringDateEntry = " 2016 09 18"
historicalDateEntry = "2015 02 25"
errorDate = "What in the heck"

In [ ]:
date = defaultDateEntry
time = ""

In [ ]:
parse("12:45").minute

In [ ]:
import datetime
answer = (datetime.datetime.now())
isinstance(answer,datetime.datetime)

### Pulling Date information

In [ ]:
#############################################
# Parse the date
#############################################


from dateutil.parser import parse
import pandas as pd
import numpy as np 
import requests
import datetime



def parse_date(var):
    """Return datetime object from string."""
    
    try:
        return np.where(isinstance(parse(var),datetime.datetime),
                 parse(var),"Error")             
    except:
        return "You entered an incorrect date.  Check your date format."

    
def dateInputCheck(parse_DateVar):
    """Check user input to retrieve date query."""
    
    return np.where(len(parse_DateVar)==0,datetime.datetime.now(),
             parse_date(parse_DateVar)) 


def gdelt_timeString(dateInputVar):
    """Convert date to GDELT string file format for query."""
    
    multiplier = dateInputVar.tolist().minute / 15
    multiple = 15 * multiplier
    queryDate = np.where(
            multiplier > 1,dateInputVar.tolist().replace(
            minute=0, second=0) + timedelta(
            minutes=multiple),
            dateInputVar.tolist().replace(
            minute=0, second=0,microsecond=0000)
            )
    
    # Check for date equality on historical query
    modifierTip = datetime.datetime.now().replace(
        hour=0,minute=0,second=0,microsecond=0
        ) == queryDate.tolist().replace(
        hour=0,minute=0,second=0,microsecond=0
        )
    
    # Based on modifier, get oldest file for historical query
    queryDate = np.where(
        modifierTip==False,
        queryDate.tolist().replace(
            hour=23,
            minute=45,
            second=00,
            microsecond=0000
            ),queryDate
        )
    
#     print modifierTip
    return queryDate.tolist().strftime("%Y%m%d%H%M%S")

#############################################
# Match parsed date to GDELT master list
#############################################

def match_date(dateString):
    """Return dataframe with GDELT data for matching date"""
    
    masterListUrl = 'http://data.gdeltproject.org/gdeltv2/masterfilelist.txt'
    directory = requests.get(masterListUrl)
    results = directory.content.split('\n')
    results = map(lambda x: x.split(' '),results)
    masterListdf = pd.DataFrame(results)
    return masterListdf[
        masterListdf[2].str.contains(
            dateString
            )==True
        ]
    


In [ ]:
resultsmatch_date(gdelt_timeString(dateInputCheck(date)))

## Munging Data: Extracting Specific Datasets or all of them

Work with the returned GDELT dataframe.  Specific whether we are pulling the `mentions`, `events`, or `gkg` date for the day or all.  

In [ ]:
results[].ix[2002]

In [ ]:
#############################################
# GDELT data download and extraction
#############################################

from StringIO import StringIO
import pandas as pd
import requests
import zipfile
import re

def downloadAndExtract(gdeltUrl):
    """Downloads and extracts GDELT zips without saving to disk"""
    
    response = requests.get(gdeltUrl, stream=True)
    zipdata = StringIO()
    zipdata.write(response.content)
    gdelt_zipfile = zipfile.ZipFile(zipdata,'r')
    name = re.search('(([\d]{4,}).*)',gdelt_zipfile.namelist()[0]).group().replace('.zip',"")
    data = gdelt_zipfile.read(name)
    return pd.read_csv(StringIO(data),delimiter='\t',header=None)
    
    
    


In [ ]:
testdf = downloadAndExtract('http://data.gdeltproject.org/gdeltv2/20150225234500.gkg.csv.zip')

# Early Pipeline to Write out R Dataframe


Ways to install
```python
pip install feather-format
```

```bash
conda install feather-format -c conda-forge
```


###  **IT WORKS!!!**

In [47]:
import feather
path = 'my_data.feather'
feather.api.write_dataframe(testdf, path)
newtestdf = feather.api.read_dataframe(path)

In [48]:
newtestdf

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,20150225234500-0,20150225234500,1,usatoday.com,http://www.usatoday.com/story/life/people/2015...,None,None,TAX_FNCACT;TAX_FNCACT_JUDGE;TAX_FNCACT_ACTRESS...,"TAX_FNCACT_DEPUTY,512;TAX_FNCACT_CHIEF,505;TAX...","4#London, London, City Of, United Kingdom#US#U...",...,"wc:170,c1.2:1,c12.1:8,c12.10:8,c12.12:1,c12.13...",http://www.gannett-cdn.com/-mm-/83c22e7929319e...,None,None,None,None,"Lindsay Lohan,56;Angeles Superior Court Judge ...",None,NaN,NaN
1,20150225234500-1,20150225234500,1,theeagle.com,http://www.theeagle.com/news/business/what-we-...,None,None,GENERAL_GOVERNMENT;ECON_INTEREST_RATES;GENERAL...,"ECON_WORLDCURRENCIES_DOLLAR,2877;TAX_POLITICAL...","1#United States#US#US#38#-97#US;3#Washington, ...",...,"wc:664,c1.2:9,c12.1:54,c12.10:92,c12.12:24,c12...",http://bloximages.chicago2.vip.townnews.com/th...,None,None,None,None,"Federal Reserve Chair Janet Yellen,121;House F...","3,things we heard,349;6,months,817;2,meetings,...",NaN,NaN
2,20150225234500-2,20150225234500,1,tbnweekly.com,http://www.tbnweekly.com/editorial/outdoors/co...,None,None,TAX_DISEASE;TAX_DISEASE_PATHOGENS;TAX_WORLDMAM...,"EDUCATION,1106;KILL,567;TAX_FNCACT_AGENT,1241;...",None,...,"wc:188,c1.3:3,c12.1:12,c12.10:19,c12.12:13,c12...",http://www.tbnweekly.com/images/fbimg_large.jpg,None,None,None,None,"Pinellas County,18;National Invasive Species A...","1000000000,of dollars every year,358;",NaN,NaN
3,20150225234500-3,20150225234500,1,journalstar.com,http://journalstar.com/ap/state/beef-plant-sit...,None,None,EDUCATION;,"EDUCATION,2969;","2#Wyoming, United States#GM#USWY#42.7475#-107....",...,"wc:516,c1.2:5,c12.1:10,c12.10:32,c12.11:1,c12....",http://bloximages.chicago2.vip.townnews.com/jo...,None,None,None,None,"Property Group,209;Josh Berger,552;Leo Hoehn,9...","2,months away,91;150,people would work at,582;...",NaN,NaN
4,20150225234500-4,20150225234500,1,kfbb.com,http://www.kfbb.com/story/28198804/representat...,None,None,TAX_FNCACT;TAX_FNCACT_SECRETARY;TAX_FNCACT_SEC...,"TAX_FNCACT_CONGRESSMAN,2172;LEADER,191;TAX_FNC...","1#Iran#US#IR#32#53#IR;2#California, United Sta...",...,"wc:571,c12.1:37,c12.10:65,c12.12:15,c12.13:15,...",None,None,None,None,None,"State John Kerry,54;Capitol Hill,95;House Fore...","12,of questions,132;2,sides over how long,1021...",NaN,NaN
5,20150225234500-5,20150225234500,1,kyma.com,http://www.kyma.com/protesters-plan-to-block-poe/,"PROTEST#8##4#Calexico, Nayarit, Mexico#MX#MX18...","PROTEST#8##4#Calexico, Nayarit, Mexico#MX#MX18...",TRAFFIC;PROTEST;GENERAL_GOVERNMENT;TAX_ETHNICI...,"BAN,779;BAN,1033;GENERAL_GOVERNMENT,366;GENERA...",1#Mexico#MX#MX#23#-102#MX;1#United States#MX#U...,...,"wc:316,c1.2:2,c12.1:17,c12.10:19,c12.12:10,c12...",http://img.youtube.com/vi/6sqbFndG4SY/0.jpg,None,None,https://youtube.com/embed/6sqbFndG4SY;https://...,609|215||The reason why we are going to close ...,"Leonardo Alvarez,221;Maria Asedas,1488","6,months,259;2,countries on Thursday,532;50000...",NaN,NaN
6,20150225234500-6,20150225234500,1,news8000.com,http://www.news8000.com/entertainment/well-mis...,None,None,TAX_FNCACT;TAX_FNCACT_WOMAN;MEDIA_MSM;LEADER;T...,"TAX_FNCACT_SECRETARY_OF_STATE,635;TAX_FNCACT_S...",None,...,"wc:445,c1.4:1,c12.1:48,c12.10:39,c12.12:11,c12...",http://www.news8000.com/image/view/-/31466204/...,None,None,https://youtube.com/user/wkbttv;,1151|25||the best day of the year!,"Amy Poehler,107;Leslie Knope,398;Ron Swanson,4...","2,were made for each,1873;",NaN,NaN
7,20150225234500-7,20150225234500,1,counton2.com,http://www.counton2.com/story/28202992/madonna...,None,None,TAX_FNCACT;TAX_FNCACT_QUEEN;,"TAX_FNCACT_QUEEN,143;","4#London, London, City Of, United Kingdom#UK#U...",...,"wc:119,c12.1:9,c12.10:8,c12.12:4,c12.13:2,c12....",http://images.worldnow.com/AP/images/6837380_G...,None,None,None,None,"For Love,230;Taylor Swift,655;Brit Awards,689","3,steps while her dancers,332;",NaN,NaN
8,20150225234500-8,20150225234500,1,myhighplains.com,http://www.myhighplains.com/story

# Leftovers; Junkyard below here

In [ ]:
results = masterListdf[masterListdf[2].str.contains(gdelt_timeString(dateInputCheck(date)))==True]

In [ ]:
results[2].reset_index().ix[0][2]

In [ ]:
results[results[2].str.contains('gkg')]

In [ ]:
gdelt_timeString(dateInputCheck(date))

In [ ]:
import re
from dateutil.parser import parse
re.search('(([\d]{4,}).*)',clean[20][-1]).group()

In [ ]:
if bool(4>3):
    print "Hello"

In [ ]:
(datetime.datetime.now().replace(hour=0,minute=0,second=0,microsecond=0)) == parse("2016 09 18" )

In [ ]:
b = dateutil.parser.parse(re.search('([\d]{4,})',clean[20][-1]).group())

In [ ]:
matchDate = re.search('([\d]{4,})',clean[20][-1]).group()

In [ ]:
def time_change(current,diff):
    date = current.replace(minute=0, second=0) + timedelta(minutes=diff)
    return date.strftime("%Y%m%d%H%M%S")
    

In [ ]:
# pulling most current daily report

import numpy as np
import datetime
from datetime import timedelta

currentTime = datetime.datetime.now()
timeDiff = currentTime.minute / 15 

query = np.where(timeDiff == 1,time_change(currentTime,diff=15),
        np.where(timeDiff == 2, time_change(currentTime,diff=30),
                 np.where(timeDiff == 3, time_change(currentTime,diff=45),
                          time_change(currentTime,diff=0))))

baseUrl = 'http://data.gdeltproject.org/gdeltv2/' + str(query) + '.export.CSV.zip'

In [ ]:
data

In [ ]:
myzipfile.namelist()

In [ ]:

import zipfile


r = requests.get(baseUrl, stream=True)

# with open('gdelt.zip', 'wb') as f:
#     f.write(r.content)
# fh = open('gdelt.zip')
# g = zipfile.ZipFile(fh)
# g.extractall()

from StringIO import StringIO
zipdata = StringIO()
zipdata.write(r.content)
myzipfile = zipfile.ZipFile(zipdata,'r')
data = myzipfile.read(str(query) + '.export.CSV')
gdeltdf = pd.read_csv(StringIO(data),delimiter='\t',header=None)


In [ ]:
gdeltdf.columns=headers.tableId.tolist()

In [ ]:
gdeltdf.SOURCEURL[((gdeltdf.ActionGeo_CountryCode =='SY')|(gdeltdf.ActionGeo_CountryCode =='IZ')) & (gdeltdf.GoldsteinScale < -4)]

In [ ]:
text = '''
GLOBALEVENTID	INTEGER	NULLABLE	This is the ID of the event that was mentioned in the article.
EventTimeDate	INTEGER	NULLABLE	This is the 15-minute timestamp (YYYYMMDDHHMMSS) when the event being mentioned was first recorded by GDELT (the DATEADDED field of the original event record).  This field can be compared against the next one to identify events being mentioned for the first time (their first mentions) or to identify events of a particular vintage being mentioned now (such as filtering for mentions of events at least one week old).
MentionTimeDate	INTEGER	NULLABLE	This is the 15-minute timestamp (YYYYMMDDHHMMSS) of the current update.  This is identical for all entries in the update file but is included to make it easier to load the Mentions table into a database.
MentionType	INTEGER	NULLABLE	This is a numeric identifier that refers to the source collection the document came from and is used to interpret the MentionIdentifier in the next column.  In essence, it specifies how to interpret the MentionIdentifier to locate the actual document.  At present, it can hold one of the following values:o 1 = WEB (The document originates from the open web and the MentionIdentifier is a fully-qualified URL that can be used to access the document on the web).o 2 = CITATIONONLY (The document originates from a broadcast, print, or other offline source in which only a textual citation is available for the document.  In this case the MentionIdentifier contains the textual citation for the document).o 3 = CORE (The document originates from the CORE archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through the CORE website).o 4 = DTIC (The document originates from the DTIC archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through the DTIC website).o 5 = JSTOR (The document originates from the JSTOR archive and the MentionIdentifier contains its DOI, suitable for accessing the original document through your JSTOR subscription if your institution subscribes to it).o 6 = NONTEXTUALSOURCE (The document originates from a textual proxy (such as closed captioning) of a non-textual information source (such as a video) available via a URL and the MentionIdentifier provides the URL of the non-textual original source.  At present, this Collection Identifier is used for processing of the closed captioning streams of the Internet Archive Television News Archive in which each broadcast is available via a URL, but the URL offers access only to the video of the broadcast and does not provide any access to the textual closed captioning used to generate the metadata.  This code is used in order to draw a distinction between URL-based textual material (Collection Identifier 1 (WEB) and URL-based non-textual material like the Television News Archive).
MentionSourceName	STRING	NULLABLE	This is a human-friendly identifier of the source of the document.  For material originating from the open web with a URL this field will contain the top-level domain the page was from.  For BBC Monitoring material it will contain “BBC Monitoring” and for JSTOR material it will contain “JSTOR.”  This field is intended for human display of major sources as well as for network analysis of information flows by source, obviating the requirement to perform domain or other parsing of the MentionIdentifier field.
MentionIdentifier	STRING	NULLABLE	This is the unique external identifier for the source document.  It can be used to uniquely identify the document and access it if you have the necessary subscriptions or authorizations and/or the document is public access.  This field can contain a range of values, from URLs of open web resources to textual citations of print or broadcast material to DOI identifiers for various document repositories.  For example, if MentionType is equal to 1, this field will contain a fully-qualified URL suitable for direct access.  If MentionType is equal to 2, this field will contain a textual citation akin to what would appear in an academic journal article referencing that document (NOTE that the actual citation format will vary (usually between APA, Chicago, Harvard, or MLA) depending on a number of factors and no assumptions should be made on its precise format at this time due to the way in which this data is currently provided to GDELT – future efforts will focus on normalization of this field to a standard citation format).  If MentionType is 3, the field will contain a numeric or alpha-numeric DOI that can be typed into JSTOR’s search engine to access the document if your institution has a JSTOR subscription.
SentenceID	INTEGER	NULLABLE	The sentence within the article where the event was mentioned (starting with the first sentence as 1, the second sentence as 2, the third sentence as 3, and so on).  This can be used similarly to the CharOffset fields below, but reports the event’s location in the article in terms of sentences instead of characters, which is more amenable to certain measures of the “importance” of an event’s positioning within an article.
Actor1CharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where Actor1 was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
Actor2CharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where Actor2 was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
ActionCharOffset	INTEGER	NULLABLE	The location within the article (in terms of English characters) where the core Action description was found.  This can be used in combination with the GKG or other analysis to identify further characteristics and attributes of the actor.  NOTE: due to processing performed on each article, this may be slightly offset from the position seen when the article is rendered in a web browser.
InRawText	INTEGER	NULLABLE	This records whether the event was found in the original unaltered raw article text (a value of 1) or whether advanced natural language processing algorithms were required to synthesize and rewrite the article text to identify the event (a value of 0).  See the discussion on the Confidence field below for more details.  Mentions with a value of “1” in this field likely represent strong detail-rich references to an event.
Confidence	INTEGER	NULLABLE	Percent confidence in the extraction of this event from this article.  See the discussion in the codebook at http://data.gdeltproject.org/documentation/GDELT-Event_Codebook-V2.0.pdf
MentionDocLen	INTEGER	NULLABLE	The length in English characters of the source document (making it possible to filter for short articles focusing on a particular event versus long summary articles that casually mention an event in passing).
MentionDocTone	FLOAT	NULLABLE	The same contents as the AvgTone field in the Events table, but computed for this particular article.  NOTE: users interested in emotional measures should use the MentionIdentifier field above to merge the Mentions table with the GKG table to access the complete set of 2,300 emotions and themes from the GCAM system.
MentionDocTranslationInfo	STRING	NULLABLE	This field is internally delimited by semicolons and is used to record provenance information for machine translated documents indicating the original source language and the citation of the translation system used to translate the document for processing.  It will be blank for documents originally in English.  At this time the field will also be blank for documents translated by a human translator and provided to GDELT in English (such as BBC Monitoring materials) – in future this field may be expanded to include information on human translation pipelines, but at present it only captures information on machine translated materials.  An example of the contents of this field might be “srclc:fra; eng:Moses 2.1.1 / MosesCore Europarl fr-en / GT-FRA 1.0”.  NOTE:  Machine translation is often not as accurate as human translation and users requiring the highest possible confidence levels may wish to exclude events whose only mentions are in translated reports, while those needing the highest-possible coverage of the non-Western world will find that these events often offer the earliest glimmers of breaking events or smaller-bore events of less interest to Western media.o SRCLC. This is the Source Language Code, representing the three-letter ISO639-2 code of the language of the original source material. o ENG.  This is a textual citation string that indicates the engine(s) and model(s) used to translate the text.  The format of this field will vary across engines and over time and no expectations should be made on the ordering or formatting of this field.  In the example above, the string “Moses 2.1.1 / MosesCore Europarl fr-en / GT-FRA 1.0” indicates that the document was translated using version 2.1.1 of the Moses   SMT platform, using the “MosesCore Europarl fr-en” translation and language models, with the final translation enhanced via GDELT Translingual’s own version 1.0 French translation and language models.  A value of “GT-ARA 1.0” indicates that GDELT Translingual’s version 1.0 Arabic translation and language models were the sole resources used for translation.  Additional language systems used in the translation pipeline such as word segmentation systems are also captured in this field such that a value of “GT-ZHO 1.0 / Stanford PKU” indicates that the Stanford Chinese Word Segmenter   was used to segment the text into individual words and sentences, which were then translated by GDELT Translingual’s own version 1.0 Chinese (Traditional or Simplified) translation and language models.
Extras	STRING	NULLABLE	This field is currently blank, but is reserved for future use to encode special additional measurements for selected material.
Add New Fields
'''

In [ ]:
from StringIO import StringIO
eventMentions = pd.read_csv(StringIO(text),delimiter='\t',header=None)

In [ ]:
eventMentions.to_csv('../../gdelt2HeaderRows/schema_csvs/GDELT_2.0_eventMentions_Column_Labels_Header_Row_Sep2016.tsv',encoding='utf-8',sep='\t')

In [ ]:
gkgdf.to_csv('../../gdelt2HeaderRows/schema_csvs/GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.tsv',encoding='utf-8',sep='\t')

In [ ]:
gkgdf.to_csv('GDELT_2.0_gdeltKnowledgeGraph_Column_Labels_Header_Row_Sep2016.csv',sep='\t',index=False,encoding='utf-8')

In [ ]:
headers.to_csv('GDELT_2.0_Events_Column_Labels_Header_Row_Sep2016.csv', index=False,encoding='utf-8')

In [ ]:
import pandas as pd
mentionsdf = pd.read_csv(StringIO(text),delimiter='\t',header=None)
mentionsdf.columns=headers.columns.tolist()